In [16]:
from bs4 import BeautifulSoup
 
 
# Reading the data inside the xml
# file to a variable under the name
# data
with open('trec.sample.xml', 'r') as f:
    data = f.read()
 
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")

In [17]:
headlines = str(Bs_data.find_all('HEADLINE')).split("</HEADLINE>")
headlines = [x.replace("\n", "") for x in headlines]
text = str(Bs_data.find_all('TEXT')).split("</TEXT>")
text = [x.replace("\n", "") for x in text]

trec = [x + y for x, y in zip(headlines, text)]
trec = [x.replace("<HEADLINE>", "") for x in trec]
trec = [x.replace("<TEXT>", "") for x in trec]


In [18]:
special_characters = ['!','"','#','$','%','&','(',')','.',"'",'?',',','*','+','/',':',';','<','=','>','@','[','\\',']','^','`','{','|','}','~','\t']
for i in special_characters : 
 trec = [x.replace(i," ") for x in trec]
origint = trec

trec = [x.split(" ") for x in trec]

print(trec[0:10])

[['', 'FT', '', '14', 'MAY', '91', '', '', '', 'CORRECTED', '', 'Jubilee', 'of', 'a', 'jet', 'that', 'did', 'what', 'it', 'was', 'designedto', 'do', 'Correction', '', 'published', '16th', 'May', '1991', '', 'appended', 'to', 'this', 'article', '', 'FRANK', '', 'it', 'flies', '', '', 'shouted', 'someone', 'at', 'Sir', 'Frank', 'Whittle', 'during', 'the', 'maidenflight', 'of', 'a', 'British', 'jet', '', '', 'Of', 'course', 'it', 'does', '', '', 'replied', 'Sir', 'Frank', '', 'whopatented', 'the', 'first', 'aircraft', 'gas', 'turbine', '', '', 'That', 's', 'what', 'it', 'was', 'bloody', 'welldesigned', 'to', 'do', '', 'wasn', 't', 'it', '', 'Exactly', '50', 'years', 'ago', 'yesterday', '', 'the', 'first', 'British', 'jet', 'made', 'a', 'brief', '17-minuteflight', 'from', 'RAF', 'Cranwell', 'in', 'Lincolnshire', '', 'To', 'celebrate', 'the', 'event', '', 'Mr', 'Eric', 'Winkle', '', 'Brown', '', 'a', '72-year-old', 'test', 'pilot', 'of', 'the', 'prototype', 'Gloster', 'Whittlejet', '', 'Mr'

In [19]:
from stemming.porter2 import stem
with open('stopwords.txt', 'r') as f:
    stop = f.read()

stop = stop.split("\n")
stop = stop[6:len(stop)]
Utrec = trec.copy()
Strec = trec.copy()
for i in range(0,len(trec)):
    Strec[i] = [word.lower() for word in Strec[i] if word.isalpha()]
    Utrec[i] = [word.lower() for word in Utrec[i] if word.isalpha()]
    Strec[i] = [stem(word) for word in Strec[i] if word not in stop]
    Utrec[i] = [stem(word) for word in Utrec[i]]

In [20]:
dict = {}
for i in range(len(Utrec)):
    for j in range(len(Utrec[i])):
        if Utrec[i][j] in Strec[i]:
            if Utrec[i][j] not in dict:
                dict[Utrec[i][j]] = []
            if Utrec[i][j] in dict:
                dict[Utrec[i][j]].append([i+1,j+1])

In [21]:
import re

def ANDquery(search):
    search = search.lower().split(" ")
    search.remove("and")
    a = stem(search[0])
    b = stem(search[1])
    res1 = set([x[0] for x in dict[a]])
    res2 = set([x[0] for x in dict[b]])
    result = list(res1.intersection(res2)) 
    result.sort(key=lambda i:int(i))
    print(result)
    return result  # result


def ANDnot(search):
    search = search.lower().split(" ")
    search.remove("and")
    search.remove('not')
    a = stem(search[0])
    b = stem(search[1])
    res1 = set([x[0] for x in dict[a]])
    res2 = set([x[0] for x in dict[b]])
    result = [a for a in res1 if a not in res2]
    return result
    
def ORquery(search):
    search = search.lower().split(" ")
    search.remove("or")
    a = stem(search[0])
    b = stem(search[1])
    res1 = set([x[0] for x in dict[a]])
    res2 = set([x[0] for x in dict[b]])
    result = list(res1.union(res2)) 
    result.sort(key=lambda i:int(i))
    print(result)
    return result  # result

def wordquery(search):
    search = stem(search.lower())
    res1 = dict[search]
    result = list(res1)
    return result

def proxsearch(search,phrase):
    dist = search.replace('#', ' ')
    print(dist)
    dist = ([re.sub('[^A-Za-z0-9]+', ' ',x) for x in dist])
    dist = "".join(dist)
    dist = dist.split(" ")
    if phrase == True:
        search.split(" ")
    while '' in dist:
        dist.remove('')
    if phrase == True:
        proximity = 1
    else:
        proximity = int(dist[0])
        
    if phrase == False:
        res1 = [x for x in dict[stem((dist[1]).lower())]]
        res2 = [x for x in dict[stem((dist[2]).lower())]]
    else:
        res1 = [x for x in dict[stem((dist[0]).lower())]]
        res2 = [x for x in dict[stem((dist[1]).lower())]]
    
    inboth = [x for x in res1 if x[0] in [x[0] for x in res2]]
    for i in range(len(inboth)):
        for j in range(len(res2)):
            if inboth[i][0] == res2[j][0]:
                if phrase == True:
                    if abs(inboth[i][1]-res2[j][1]) <= proximity:
                        return inboth[i], res2[j]
                if abs(inboth[i][1]-res2[j][1]) <= proximity:
                    return inboth[i], res2[j]
                
    
proxsearch("income taxes",True)

income taxes


([65, 201], [65, 202])